In [1]:
import bayesflow as bf
from AmortizedMixture import *

/Users/simonkucharsky/projects/bayesflow/amortized-mixture/.venv/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
@prior
def normal_mixture_prior(num_classes=2):
    return {
        'class_probabilities': np.random.dirichlet([1 for _ in range(num_classes)]),
        'class_parameters': np.sort(np.random.normal(size=num_classes))
    }

In [29]:
@simulator
def normal_mixture_simulator(params, num_obs=1000, num_classes=2):
    pi, mu = params.values()
    z = np.random.choice(a = [i for i in range(num_classes)], size = num_obs, replace = True, p=pi)
    loc = mu[z]
    y = np.random.normal(loc=loc, size=num_obs)
    return {
        "latents": z,
        "observables": np.c_[y]
    }


In [30]:
model = bf.simulation.GenerativeModel(prior=normal_mixture_prior, simulator=normal_mixture_simulator)

INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, )
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, )
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


In [31]:
configurator = DefaultJointMixtureConfigurator(num_classes=2)

In [32]:
data = model(3)


In [33]:
conf_data = configurator(data)

In [34]:
conf_data['posterior_inputs']['summary_conditions'].shape

(3, 1000, 1)

In [35]:
net = bf.inference_networks.InvertibleNetwork(num_params=1)

In [61]:
d = np.array([[0, 2, 0], [0, 2, 3]])
d.shape

(2, 3)

In [62]:
net(d.astype(np.float32), [[0], [0]])

(<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[0.],
        [0.]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 0.], dtype=float32)>)